In [1]:
import os
from functools import partial
import json
import csv

import geopandas as gpd
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook

from gerrychain import (
    Election,
    Graph,
    MarkovChain,
    Partition,
    accept,
    constraints,
    updaters,
)

from gerrychain.metrics import efficiency_gap, mean_median, partisan_gini
from gerrychain.proposals import recom
from gerrychain.updaters import cut_edges
from gerrychain.tree import recursive_tree_part

In [2]:
graph_path = "./missouri.json"
graph = Graph.from_json(graph_path)
with open (graph_path, "r") as myfile:
    data=myfile.readlines()
graph_dict = json.loads(data[0])

In [29]:
with open("test.csv", mode="w") as test:
    data_writer = csv.writer(test, delimiter=",", quotechar='"', quoting=csv.QUOTE_MINIMAL)   
    
    data_writer.writerow(["dem_votes", "rep_votes", "prec_pop", "node_num"])

    for n in graph.nodes:
        data_writer.writerow([graph.nodes[n]["SEN16D"],
                  graph.nodes[n]["SEN16R"],
                  graph.nodes[n]["TOTPOP"],
                  graph.nodes[n]["PrcncID"]])

In [6]:
graph.nodes[2]

{'boundary_node': False,
 'area': 117386667.93554994,
 'STATEFP10': '29',
 'COUNTYFP10': '003',
 'VTDST10': '21',
 'GEOID10': '2900321',
 'VTDI10': 'P',
 'NAME10': 'Clay',
 'NAMELSAD10': 'Clay Voting District',
 'LSAD10': 'V2',
 'MTFCC10': 'G5240',
 'FUNCSTAT10': 'S',
 'ALAND10': 117237507,
 'AWATER10': 161472,
 'INTPTLAT10': '+40.0917961',
 'INTPTLON10': '-094.9668992',
 'POP100': 259,
 'VAP': 183,
 'COUNTY': 'ANDREW',
 'PR_RV08': 79.0,
 'PR_DV08': 27.0,
 'PR_OTHV08': 2.0,
 'USH_DV08': 25.0,
 'USH_RV08': 81.0,
 'GOV_DV08': 42.0,
 'GOV_RV08': 63.0,
 'LG_RV08': 78.0,
 'LG_DV08': 21.0,
 'SS_DV08': 48.0,
 'SS_RV08': 53.0,
 'TR_OTHV08': 2.0,
 'TR_DV08': 22.0,
 'TR_RV08': 81.0,
 'AG_RV08': 69.0,
 'AG_DV08': 29.0,
 'GOV_OTHV08': 1.0,
 'LG_OTH08': 4.0,
 'SS_OTHV08': 4.0,
 'SUMVAP': 183,
 'VAPSHARE': 1.0,
 'P_08': 0.25471699238,
 'USH_08': 0.23584905267,
 'GOV_08': 0.40000000596,
 'TR_08': 0.21359223127,
 'AG_08': 0.29591837525,
 'LG_08': 0.21212121844,
 'AV': 0.268699646,
 'NDV': 29.019561767

In [7]:
pop_count = 0

for i in graph.nodes:
    pop_count += graph.nodes[i]["POP100"]
    
print(pop_count)

5843434


In [9]:
%%time
num_dist = 8

# Exercise: Compute exact population from your data.
pop = pop_count

my_updaters = {
    "population": updaters.Tally("TOTPOP", alias="population"),
    "cut_edges": cut_edges,
    "SS08": Election("SS08", {"democratic":"SS_DV08","republican":"SS_RV08"})
}


new_plan = recursive_tree_part(graph,
                               range(num_dist),
                               pop/num_dist,
                               "POP100",
                               0.004,
                               1)
initial_partition = Partition(graph,
                              new_plan,
                              my_updaters)

CPU times: user 8.68 s, sys: 154 ms, total: 8.84 s
Wall time: 8.97 s


In [10]:
pop_prec = []
reps_prec = []
dems_prec = []

for n in tqdm_notebook(graph.nodes):
    pop_prec.append(graph.nodes[n]["POP100"])
    reps_prec.append(graph.nodes[n]["SS_RV08"])
    dems_prec.append(graph.nodes[n]["SS_DV08"])

In [10]:
(dems_prec[0] - reps_prec[0]) / pop_prec[0]

-0.38460756552620307

In [14]:
ascending_DPC = []

for i in range(len(pop_prec)):
    if pop_prec[i] != 0:
        ascending_DPC.append([(dems_prec[i] - reps_prec[i]) / pop_prec[i], i])

In [12]:
len(ascending_DPC)

4484

In [15]:
ascending_DPC[:10]

[[0.03360957642725598, 0],
 [0.041666666666666664, 1],
 [-0.019305019305019305, 2],
 [-0.00936768149882904, 3],
 [0.019021739130434784, 4],
 [0.028368794326241134, 5],
 [0.06408094435075885, 6],
 [0.02258064516129032, 7],
 [-0.02422145328719723, 8],
 [0.11475409836065574, 9]]

In [16]:
ascending_DPC.sort()

In [20]:
ascending_DPC[:10]

[[-0.2490566037735849, 74],
 [-0.24871794871794872, 73],
 [-0.21524663677130046, 64],
 [-0.20903954802259886, 1706],
 [-0.2, 65],
 [-0.19480519480519481, 67],
 [-0.19318181818181818, 4260],
 [-0.18882978723404256, 1953],
 [-0.15717092337917485, 4687],
 [-0.156794425087108, 1707]]

In [18]:
descending_DPC = ascending_DPC.copy()
descending_DPC.reverse()

In [21]:
descending_DPC[:10]

[[1.5, 2420],
 [0.8753246753246753, 2299],
 [0.8181818181818182, 4524],
 [0.7865168539325843, 4556],
 [0.7742663656884876, 1491],
 [0.6993006993006993, 4523],
 [0.6972477064220184, 4631],
 [0.6953807811566667, 1443],
 [0.6771561771561772, 4492],
 [0.6288556424503586, 1486]]

In [87]:
ascending_DPC[:10]

[[-33.47983173963706, 1362],
 [-23.7976727460307, 1829],
 [-20.22352546835022, 1670],
 [-14.031384817544831, 1252],
 [-10.502181943758941, 1313],
 [-6.687244415233582, 1311],
 [-6.660755182224414, 1169],
 [-6.371563798385555, 1351],
 [-5.0321267703814705, 1689],
 [-3.976267330082986, 713]]

In [22]:
I = pop_count/8
print(I)

730429.25


In [24]:
pop_counter = 0
dem_qdist_1 = []
dem_qdist_2 = []

for j in descending_DPC:
    if pop_counter <= I:
        dem_qdist_1.append(j[1])
        pop_counter += graph.nodes[j[1]]["POP100"]
    elif pop_counter > I and pop_counter <= 2*I:
        dem_qdist_2.append(j[1])
        pop_counter += graph.nodes[j[1]]["POP100"]
    else:
        break

In [25]:
dem_votes_qdist_1 = 0
rep_votes_qdist_1 = 0
total_pop_qdist_1 = 0
dem_votes_qdist_2 = 0
rep_votes_qdist_2 = 0
total_pop_qdist_2 = 0

for n in dem_qdist_1:
    dem_votes_qdist_1 += graph.nodes[n]["SS_DV08"]
    rep_votes_qdist_1 += graph.nodes[n]["SS_RV08"]
    total_pop_qdist_1 += graph.nodes[n]["POP100"]
    
for n in dem_qdist_2:
    dem_votes_qdist_2 += graph.nodes[n]["SS_DV08"]
    rep_votes_qdist_2 += graph.nodes[n]["SS_RV08"]
    total_pop_qdist_2 += graph.nodes[n]["POP100"]
    
votes_qdist_1 = rep_votes_qdist_1 + dem_votes_qdist_1
votes_qdist_2 = rep_votes_qdist_2 + dem_votes_qdist_2

rep_vs_qdist_1 = rep_votes_qdist_1 / votes_qdist_1
rep_vs_qdist_2 = rep_votes_qdist_2 / votes_qdist_2

print(rep_vs_qdist_1, rep_vs_qdist_2)

overall_rep_vs = (rep_votes_qdist_1 + rep_votes_qdist_2) / (votes_qdist_1 + votes_qdist_2)
print(overall_rep_vs)

0.11239639141363246 0.27391988415010965
0.18822666239099747


In [26]:
pop_counter = 0
rep_qdist_1 = []
rep_qdist_2 = []

for j in ascending_DPC:
    if pop_counter <= I:
        rep_qdist_1.append(j[1])
        pop_counter += graph.nodes[j[1]]["POP100"]
    elif pop_counter > I and pop_counter <= 2*I:
        rep_qdist_2.append(j[1])
        pop_counter += graph.nodes[j[1]]["POP100"]
    else:
        break

In [27]:
dem_votes_qdist_1 = 0
rep_votes_qdist_1 = 0
total_pop_qdist_1 = 0
dem_votes_qdist_2 = 0
rep_votes_qdist_2 = 0
total_pop_qdist_2 = 0

for n in rep_qdist_1:
    dem_votes_qdist_1 += graph.nodes[n]["SS_DV08"]
    rep_votes_qdist_1 += graph.nodes[n]["SS_RV08"]
    total_pop_qdist_1 += graph.nodes[n]["POP100"]
    
for n in rep_qdist_2:
    dem_votes_qdist_2 += graph.nodes[n]["SS_DV08"]
    rep_votes_qdist_2 += graph.nodes[n]["SS_RV08"]
    total_pop_qdist_2 += graph.nodes[n]["POP100"]
    
votes_qdist_1 = rep_votes_qdist_1 + dem_votes_qdist_1
votes_qdist_2 = rep_votes_qdist_2 + dem_votes_qdist_2

rep_vs_qdist_1 = rep_votes_qdist_1 / votes_qdist_1
rep_vs_qdist_2 = rep_votes_qdist_2 / votes_qdist_2

print(rep_vs_qdist_1, rep_vs_qdist_2)

overall_rep_vs = (rep_votes_qdist_1 + rep_votes_qdist_2) / (votes_qdist_1 + votes_qdist_2)
print(overall_rep_vs)

0.5481209868370575 0.47443044583197963
0.5128520526359723
